## INTERACTIVE MATHS
- Tubes NumeroUno

**Import Library Yang Digunakan**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import mediapipe as mp

**Inisialisasi Hand Landmarks**

In [2]:
# Inisialisasi MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=3, min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

**Webcam**

In [3]:
# Test Webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
		print("Error: Could not open webcam.")
		exit()

while cap.isOpened():
		ret, frame = cap.read()
		if not ret:
				print("Error: Could not read frame.")
				break
		
		# Konversi BGR ke RGB
		frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

		# Deteksi tangan
		results = hands.process(frame_rgb)
		if results.multi_hand_landmarks:
				for hand_landmarks in results.multi_hand_landmarks:
						mp_drawing.draw_landmarks(
								frame,
								hand_landmarks,
								mp_hands.HAND_CONNECTIONS,
								mp_drawing_styles.get_default_hand_landmarks_style(),
								mp_drawing_styles.get_default_hand_connections_style())

		# Tampilkan frame dengan deteksi tangan
		cv2.imshow('Webcam', frame)

		if cv2.waitKey(1) & 0xFF == ord('q'):
				break

cap.release()
cv2.destroyAllWindows()


**Deteksi Jari**

In [4]:
# Daftar Indeks Landmark Tangan (1-10)
JEMPOL = [1, 2, 3, 4]  # Jari Jempol
TELUNJUK = [5, 6, 7, 8]  # Jari Telunjuk
JARI_TENGAH = [9, 10, 11, 12]  # Jari Tengah
JARI_MANIS = [13, 14, 15, 16]  # Jari Manis
JARI_KELINGKING = [17, 18, 19, 20]  # Jari Kelingking

**Deteksi Tangan Kiri atau Kanan**

In [5]:
def detect_finger_number(hand_landmarks, handedness):
	fingers = []
	
	# Thumb detection - different logic for left and right hands
	thumb_tip = hand_landmarks.landmark[JEMPOL[-1]]
	thumb_base = hand_landmarks.landmark[JEMPOL[0]]
	finger_offset = 0.06 
	if handedness == 'Right':
		fingers.append(thumb_tip.x < (thumb_base.x - finger_offset))
	else:
		fingers.append(thumb_tip.x > (thumb_base.x + finger_offset))
	
	# Other fingers
	finger_lists = [TELUNJUK, JARI_TENGAH, JARI_MANIS, JARI_KELINGKING]
	
	for finger in finger_lists:
		tip = hand_landmarks.landmark[finger[-1]]
		base = hand_landmarks.landmark[finger[0]]
		fingers.append(tip.y < (base.y - finger_offset))
	
	return sum(fingers)

# Modify webcam loop
cap = cv2.VideoCapture(0)

while cap.isOpened():
	ret, frame = cap.read()
	if not ret:
		print("Error: Could not read frame.")
		break
	
	frame = cv2.flip(frame, 1)
	frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	
	results = hands.process(frame_rgb)

	if results.multi_hand_landmarks:
		total_fingers = 0
		for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
			handedness = results.multi_handedness[idx].classification[0].label
			count = detect_finger_number(hand_landmarks, handedness)
			total_fingers += count
			
			mp_drawing.draw_landmarks(
				frame,
				hand_landmarks,
				mp_hands.HAND_CONNECTIONS,
				mp_drawing_styles.get_default_hand_landmarks_style(),
				mp_drawing_styles.get_default_hand_connections_style()
			)
		
		# Limit total detected fingers between 0-10
		total_fingers = min(10, total_fingers)
		
		# Display the total finger count
		cv2.putText(
			frame, 
			str(total_fingers), 
			(50, 150), 
			cv2.FONT_HERSHEY_SIMPLEX, 
			4,
			(0, 255, 0),
			4
		)
	
	cv2.imshow('Hand Number Detection', frame)
	
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

cap.release()
cv2.destroyAllWindows()
